In [65]:
import yfinance as yf
import pandas as pd
import warnings
import plotly.express as px

from optimisation_markowitz import MarkowitzOptimization
from optimisation_simm import SingleFactorModel, SingleFactorModelOptimization
from utils import Utils
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
start_date = "2023-04-01"
end_date = "2024-04-01"
annualized_factor = 252

We will use S&P500 Information Technology Index as benchmark to evaluate the performance with Single Index Market Model and Constant Correlation Model. The return over four years will be used as the market return in the formula

In [67]:
# Get close data for S&P 500 Information Technology Index
market_index_data = Utils.get_historical_data("^SP500-45", start_date, end_date)

# calculate simple return for market index
market_index_simple_return = Utils.calculate_simple_return(market_index_data)

# calculate daily market data returns.
# use that to calculate expected returns and variance
market_index_daily_returns = Utils.calculate_daily_returns(market_index_data)
market_index_data_expected_return = market_index_daily_returns.mean()
market_index_data_variance = market_index_daily_returns.var()

print(f"Market Index Simple Return from {start_date} - {end_date} in %: ", market_index_simple_return * 100)
print("Market Index Expected daily Return in %: ", market_index_data_expected_return * 100)
print("Market Index daily Variance in %: ", market_index_data_variance * 100)

[*********************100%%**********************]  1 of 1 completed
Market Index Simple Return from 2023-04-01 - 2024-04-01 in %:  44.858555542505044
Market Index Expected daily Return in %:  0.1563335685653037
Market Index daily Variance in %:  0.013689625923552962


For markowitz model, we will use 3-Month US Treasury Bill as risk-free rate. Since we are holding the portfolio for 1 year, we will take the last value as the risk-free rate.

In [68]:
risk_free_rate_df = yf.download("^IRX", start_date, end_date)
risk_free_rate = risk_free_rate_df['Adj Close'].iloc[-1] / 100
risk_free_rate

[*********************100%%**********************]  1 of 1 completed


0.05203000068664551

# Technology Sector ETFs

We will now download the data for US Technology Equity ETFs on which we will run our strategies. We will use the following top 5 ETFs based on total assets:
1. Vanguard Information Technology ETF (VGT)
2. Technology Select Sector SPDR Fund (XLK)
3. VanEck Semiconductor ETF (SMH)
4. iShares US Technology ETF (IYW)
5. iShares Semiconductor ETF (SOXX)

Source: https://etfdb.com/etfdb-category/technology-equities/ 

In [69]:
# get the data
vgt_data = Utils.get_historical_data("VGT", start_date, end_date)
xlk_data = Utils.get_historical_data("XLK", start_date, end_date)
smh_data = Utils.get_historical_data("SMH", start_date, end_date)
iyt_data = Utils.get_historical_data("IYW", start_date, end_date)
soxx_data = Utils.get_historical_data("SOXX", start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [70]:
# check for any missing data
print(market_index_data.isnull().sum())
print(vgt_data.isnull().sum())
print(xlk_data.isnull().sum())
print(smh_data.isnull().sum())
print(iyt_data.isnull().sum())
print(soxx_data.isnull().sum())

Adj Close        0
simple_return    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64
Adj Close    0
dtype: int64


We will next calculate the simple return over the period of 1 years for those ETFs. We will use the following formula to calculate the return:

$$
\text{Simple Return} = \frac{P_{t} - P_{t-1}}{P_{t-1}}
$$

Where:
- $P_{t}$ is the price at time $t$
- $P_{t-1}$ is the price at time $t-1$
- $t$ is the current time

In [71]:

vgt_simple_return = Utils.calculate_simple_return(vgt_data)
xlk_simple_return = Utils.calculate_simple_return(xlk_data)
smh_simple_return = Utils.calculate_simple_return(smh_data)
iyt_simple_return = Utils.calculate_simple_return(iyt_data)
soxx_simple_return = Utils.calculate_simple_return(soxx_data)

simple_returns_dict = {
    "VGT": vgt_simple_return,
    "XLK": xlk_simple_return,
    "SMH": smh_simple_return,
    "IYT": iyt_simple_return,
    "SOXX": soxx_simple_return
}

for ticker, simple_return in simple_returns_dict.items():
    print(f"{ticker} -> Simple Return from {start_date} - {end_date} in %: ", simple_return * 100)

VGT -> Simple Return from 2023-04-01 - 2024-04-01 in %:  37.26376098928226
XLK -> Simple Return from 2023-04-01 - 2024-04-01 in %:  39.258222440423566
SMH -> Simple Return from 2023-04-01 - 2024-04-01 in %:  72.98801691288277
IYT -> Simple Return from 2023-04-01 - 2024-04-01 in %:  46.16350385840067
SOXX -> Simple Return from 2023-04-01 - 2024-04-01 in %:  57.11977609580361


In [72]:
# plot the simple retruns using plotly for all stocks along with market index
fig = px.line(title="Simple Returns for Technology Sector ETFs and Market Index")
fig.add_scatter(x=vgt_data.index, y=vgt_data['simple_return'], mode='lines', name='VGT')
fig.add_scatter(x=xlk_data.index, y=xlk_data['simple_return'], mode='lines', name='XLK')
fig.add_scatter(x=smh_data.index, y=smh_data['simple_return'], mode='lines', name='SMH')
fig.add_scatter(x=iyt_data.index, y=iyt_data['simple_return'], mode='lines', name='IYT')
fig.add_scatter(x=soxx_data.index, y=soxx_data['simple_return'], mode='lines', name='SOXX')
fig.add_scatter(x=market_index_data.index, y=market_index_data['simple_return'], mode='lines', name='Market Index')
fig.show()

On observing the simple returns, we see that VanEck Semiconductor ETF (SMH) had the highest return. This could be due to the fact that it's expense ratio is among the lowest along with it's holdings in NVIDIA and Taiwan Semiconductor Manufacturing Company.

We will next calculate daily returns and observe any spikes in the returns.

In [73]:
vga_daily_returns = Utils.calculate_daily_returns(vgt_data)
xlk_daily_returns = Utils.calculate_daily_returns(xlk_data)
smh_daily_returns = Utils.calculate_daily_returns(smh_data)
iyt_daily_returns = Utils.calculate_daily_returns(iyt_data)
soxx_daily_returns = Utils.calculate_daily_returns(soxx_data)


daily_returns_df = pd.DataFrame([vga_daily_returns, xlk_daily_returns, smh_daily_returns, iyt_daily_returns, soxx_daily_returns]).T 
daily_returns_df.columns = ["VGT", "XLK", "SMH", "IYT", "SOXX"]

In [74]:
# we can now plot the daily returns for all the stocks in sub plots
fig = make_subplots(rows=3, cols=2, subplot_titles=("VGT", "XLK", "SMH", "IYT", "SOXX", "Market Index"))

fig.add_scatter(x=vgt_data.index, y=vga_daily_returns, mode='lines', name='VGT', row=1, col=1)
fig.add_scatter(x=xlk_data.index, y=xlk_daily_returns, mode='lines', name='XLK', row=1, col=2)
fig.add_scatter(x=smh_data.index, y=smh_daily_returns, mode='lines', name='SMH', row=2, col=1)
fig.add_scatter(x=iyt_data.index, y=iyt_daily_returns, mode='lines', name='IYT', row=2, col=2)
fig.add_scatter(x=soxx_data.index, y=soxx_daily_returns, mode='lines', name='SOXX', row=3, col=1)
fig.add_scatter(x=market_index_data.index, y=market_index_daily_returns, mode='lines', name='Market Index', row=3, col=2)

fig.update_layout(title_text="Daily Returns for Tech Stocks and Market Index", height=1000, width=1000)
fig.show()


In [75]:
cov_matrix = daily_returns_df.cov()
cov_matrix

,VGT,XLK,SMH,IYT,SOXX
VGT,0.000125,0.000122,0.000171,0.000127,0.000169
XLK,0.000122,0.000122,0.000167,0.000125,0.000164
SMH,0.000171,0.000167,0.000298,0.000173,0.000294
IYT,0.000127,0.000125,0.000173,0.000133,0.000169
SOXX,0.000169,0.000164,0.000294,0.000169,0.000304


In [76]:
corr_matrix = daily_returns_df.corr()
corr_matrix

,VGT,XLK,SMH,IYT,SOXX
VGT,1.000000,0.991351,0.883926,0.982889,0.865784
XLK,0.991351,1.000000,0.874606,0.982606,0.851024
SMH,0.883926,0.874606,1.000000,0.866822,0.975605
IYT,0.982889,0.982606,0.866822,1.000000,0.839290
SOXX,0.865784,0.851024,0.975605,0.839290,1.000000


In [77]:
# plot correlation using plotly with cell values and centre aligned title
fig = px.imshow(corr_matrix, color_continuous_scale='RdBu', title="Correlation Matrix between returns of Technology Sector ETFs")
fig.update_layout(title_x=0.5)
fig.show()

As expected, we observe that Technology Sector ETFs are highly correlated to each other.

In [78]:
# standard deviations between daily returns
std_dev = daily_returns_df.std()
std_dev

VGT     0.011188
XLK     0.011044
SMH     0.017272
IYT     0.011536
SOXX    0.017427
dtype: float64

In [79]:
# daily mean return
mean_daily_return = daily_returns_df.mean()
mean_daily_return

VGT     0.001340
XLK     0.001397
SMH     0.002359
IYT     0.001598
SOXX    0.001974
dtype: float64

## Optimal tangency portfolio using Markowitz model 

In [80]:
# Run the Markowitz Portfolio Optimization
simple_returns_list = list(simple_returns_dict.values())
markowitz = MarkowitzOptimization(simple_returns_list, cov_matrix, risk_free_rate)


In [109]:
# find the weights of the portfolio with short shelling
markowitz_weights_with_ss = markowitz.find_tangency_portfolio_with_short_selling()

In [110]:
# create the wieghts dictionary
markowitz_weights_with_ss_dict = {
    "VGT": markowitz_weights_with_ss[0],
    "XLK": markowitz_weights_with_ss[1],
    "SMH": markowitz_weights_with_ss[2],
    "IYT": markowitz_weights_with_ss[3],
    "SOXX": markowitz_weights_with_ss[4]
}

for ticker, weight in markowitz_weights_with_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights with Short Selling in %: ", weight * 100)

VGT -> Markowitz Weights with Short Selling in %:  -500.0
XLK -> Markowitz Weights with Short Selling in %:  -20.587920209334058
SMH -> Markowitz Weights with Short Selling in %:  500.0
IYT -> Markowitz Weights with Short Selling in %:  500.0
SOXX -> Markowitz Weights with Short Selling in %:  -379.412079790666


In [111]:
# find the weights of the portfolio without short shelling
markowitz_weights_without_ss = markowitz.find_tangency_portfolio_no_short_selling()

In [112]:
# weights dictionary
markowitz_weights_without_ss_dict = {
    "VGT": markowitz_weights_without_ss[0],
    "XLK": markowitz_weights_without_ss[1],
    "SMH": markowitz_weights_without_ss[2],
    "IYT": markowitz_weights_without_ss[3],
    "SOXX": markowitz_weights_without_ss[4]
}

for ticker, weight in markowitz_weights_without_ss_dict.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling in %: ", weight * 100)

VGT -> Markowitz Weights without Short Selling in %:  0.0
XLK -> Markowitz Weights without Short Selling in %:  3.2151766310407442e-12
SMH -> Markowitz Weights without Short Selling in %:  79.17791738459209
IYT -> Markowitz Weights without Short Selling in %:  20.82208261542255
SOXX -> Markowitz Weights without Short Selling in %:  0.0


In [114]:
expected_return_with_ss, volatility_with_ss = Utils.calculate_portfolio_return_and_risk(markowitz_weights_with_ss, simple_returns_list, cov_matrix)
print("Volatility with Short Selling in %: ", volatility_with_ss * 100)
print("Expected Portfolio Return in %: ", expected_return_with_ss * 100)

Volatility with Short Selling in %:  2.8489454263764267
Expected Portfolio Return in %:  184.6370169415084


In [137]:
# calculate sharpe ratio
portfolio_mean_return_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling in %: ", portfolio_mean_return_ss * 100)

sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_ss, volatility_with_ss, risk_free_rate)
print("Sharpe Ratio with Short Selling: ", sharpe_ratio_with_ss)

Mean Portfolio Return with Short Selling in %:  8.427224856763681
Sharpe Ratio with Short Selling:  1.0721274950547954


In [115]:
expected_return_without_ss, volatility_without_ss = Utils.calculate_portfolio_return_and_risk(markowitz_weights_without_ss, simple_returns_list, cov_matrix)
print("Volatility without Short Selling in %: ", volatility_without_ss * 100)
print("Expected Portfolio Return without Short Selling in %: ", expected_return_without_ss * 100)

Volatility without Short Selling in %:  1.5803000889711756
Expected Portfolio Return without Short Selling in %:  67.40259464350565


In [116]:
# calculate sharpe ratio
portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return without Short Selling in %: ", portfolio_mean_return_without_ss * 100)

sharpe_ratio_without_ss = Utils.calculate_sharpe_ratio(portfolio_mean_return_without_ss, volatility_without_ss, risk_free_rate)
print("Sharpe Ratio without Short Selling: ", sharpe_ratio_without_ss)

Mean Portfolio Return without Short Selling in %:  3.4937896111046607
Sharpe Ratio without Short Selling:  0.3677665763753523


In [117]:
# we are interested in finding optimal weights when an asset can hold 30% of the portfolio max
markowitz_weights_without_ss_pct_constraint = markowitz.find_tangency_portfolio_no_short_selling_and_max_weight(0.3)

In [118]:
markowitz_weights_dict_without_ss_pct_constraint = {
    "VGT": markowitz_weights_without_ss_pct_constraint[0],
    "XLK": markowitz_weights_without_ss_pct_constraint[1],
    "SMH": markowitz_weights_without_ss_pct_constraint[2],
    "IYT": markowitz_weights_without_ss_pct_constraint[3],
    "SOXX": markowitz_weights_without_ss_pct_constraint[4]
}

for ticker, weight in markowitz_weights_dict_without_ss_pct_constraint.items():
    print(f"{ticker} -> Markowitz Weights without Short Selling and 30% constraint in %: ", weight * 100)

VGT -> Markowitz Weights without Short Selling and 30% constraint in %:  5.561062677539843
XLK -> Markowitz Weights without Short Selling and 30% constraint in %:  30.00000000005351
SMH -> Markowitz Weights without Short Selling and 30% constraint in %:  30.000000000184922
IYT -> Markowitz Weights without Short Selling and 30% constraint in %:  30.00000000007661
SOXX -> Markowitz Weights without Short Selling and 30% constraint in %:  4.438937322145114


In [119]:
expected_return_without_ss_and_constraint, volatility_without_ss_with_constraint = Utils.calculate_portfolio_return_and_risk(markowitz_weights_without_ss_pct_constraint, simple_returns_list, cov_matrix)
print("Volatility with Short Selling and Constraint in %: ", volatility_without_ss_with_constraint * 100)
print("Expected Portfolio Return with Short Selling and Constraint in %: ", expected_return_without_ss_and_constraint * 100)

Volatility with Short Selling and Constraint in %:  1.288293726963541
Expected Portfolio Return with Short Selling and Constraint in %:  52.13069512776842


In [120]:
# find the Sharpe Ratio
portfolio_mean_return_with_ss_with_constraint = Utils.calculate_portfolio_mean_return(markowitz_weights_without_ss_pct_constraint, mean_daily_return, annualized_factor)
print("Mean Portfolio Return with Short Selling and Constraint in %: ", portfolio_mean_return_with_ss_with_constraint * 100)

sharpe_no_ss_with_constraint = Utils.calculate_sharpe_ratio(portfolio_mean_return_with_ss_with_constraint, volatility_without_ss_with_constraint, risk_free_rate)
print("Sharpe Ratio with Short Selling and Constraint: ", sharpe_no_ss_with_constraint)

Mean Portfolio Return with Short Selling and Constraint in %:  2.8072243737550013
Sharpe Ratio with Short Selling and Constraint:  0.29193362036247733


## Single Index Market Model

We will now compare results from markowitz model with single index market model. We will use S&P500 Information Technology Index as benchmark to evaluate the performance with Single Index Market Model.

From this point, we will run `SingleFactorModel` class over all ETFs calculating beta, alpha and tau. We will then calculate the expected return using the formula:

$$
u_i = \alpha_i + \beta_i \cdot u_m

In [121]:
def get_simm_params(asset_return: pd.DataFrame, market_return: pd.DataFrame):
  model = SingleFactorModel(asset_return, market_return)
  model.fit()
  
  params = model.get_params()
  alpha = params.get("alpha").get("value")
  beta = params.get("beta").get("value")
  tau = params.get("tau").get("value")
  
  return (alpha, beta, tau)
  

In [122]:
# run SIMM for params
vgt_alpha, vgt_beta, vgt_tau = get_simm_params(vga_daily_returns, market_index_daily_returns)
xlk_alpha, xlk_beta, xlk_tau = get_simm_params(xlk_daily_returns, market_index_daily_returns)
smh_alpha, smh_beta, smh_tau = get_simm_params(smh_daily_returns, market_index_daily_returns)
iyt_alpha, iyt_beta, iyt_tau = get_simm_params(iyt_daily_returns, market_index_daily_returns)
soxx_alpha, soxx_beta, soxx_tau = get_simm_params(soxx_daily_returns, market_index_daily_returns)

In [123]:
# get expected returns using alpha, beta and expected market return
vgt_expected_return = vgt_alpha + (vgt_beta * market_index_data_expected_return)
xlk_expected_return = xlk_alpha + (xlk_beta * market_index_data_expected_return)
smh_expected_return = smh_alpha + (smh_beta * market_index_data_expected_return)
iyt_expected_return = iyt_alpha + (iyt_beta * market_index_data_expected_return)
soxx_expected_return = soxx_alpha + (soxx_beta * market_index_data_expected_return)

In [124]:
# we can now pack the expected returns into a dictionary along with beta and tau
expected_returns_dict = {
    "VGT": vgt_expected_return,
    "XLK": xlk_expected_return,
    "SMH": smh_expected_return,
    "IYT": iyt_expected_return,
    "SOXX": soxx_expected_return
}

alpha_dict = {
    "VGT": vgt_alpha,
    "XLK": xlk_alpha,
    "SMH": smh_alpha,
    "IYT": iyt_alpha,
    "SOXX": soxx_alpha
}

beta_dict = {
    "VGT": vgt_beta,
    "XLK": xlk_beta,
    "SMH": smh_beta,
    "IYT": iyt_beta,
    "SOXX": soxx_beta
}

tau_dict = {
    "VGT": vgt_tau,
    "XLK": xlk_tau,
    "SMH": smh_tau,
    "IYT": iyt_tau,
    "SOXX": soxx_tau
}

for i in range(len(expected_returns_dict.keys())):
  print(f"{ list(expected_returns_dict.keys())[i] } -> Expected Daily Return in %: ", list(expected_returns_dict.values())[i] * 100)
  print(f"{ list(alpha_dict.keys())[i] } -> Alpha: ", list(alpha_dict.values())[i])
  print(f"{ list(beta_dict.keys())[i] } -> Beta: ", list(beta_dict.values())[i])
  print(f"{ list(tau_dict.keys())[i] } -> Tau: ", list(tau_dict.values())[i])
  print()

VGT -> Expected Daily Return in %:  0.1340205684478544
VGT -> Alpha:  -0.00012983770346941813
VGT -> Beta:  0.9403248460575473
VGT -> Tau:  0.0020368479633535107

XLK -> Expected Daily Return in %:  0.13968243208637823
XLK -> Alpha:  -6.718229614961784e-05
XLK -> Beta:  0.9364633779224805
XLK -> Tau:  0.001390126993822133

SMH -> Expected Daily Return in %:  0.2359472140425165
SMH -> Alpha:  0.0003236418568278154
SMH -> Beta:  1.3022348957299865
SMH -> Tau:  0.008150764625956512

IYT -> Expected Daily Return in %:  0.15978213420353785
IYT -> Alpha:  9.123092402409436e-05
IYT -> Beta:  0.9637024420522652
IYT -> Tau:  0.0024403067706091205

SOXX -> Expected Daily Return in %:  0.19738984714288643
SOXX -> Alpha:  2.0177119588102543e-05
SOXX -> Beta:  1.2497132700099867
SOXX -> Tau:  0.009501296182334784



In [125]:
# run the SIMM optimization
expected_returns_list = list(expected_returns_dict.values())
beta_list = list(beta_dict.values())
tau_list = list(tau_dict.values())
alpha_list = list(alpha_dict.values())

simm_optimization = SingleFactorModelOptimization(expected_returns_list, beta_list, tau_list, market_index_data_variance, risk_free_rate)

In [126]:
# find the optimal weights when short selling is allowed
simm_weights_with_ss = simm_optimization.find_weights_short_selling_allowed()

In [127]:
simm_weights_with_ss_dict = {
  "VGT": simm_weights_with_ss[0],
  "XLK": simm_weights_with_ss[1],
  "SMH": simm_weights_with_ss[2],
  "IYT": simm_weights_with_ss[3],
  "SOXX": simm_weights_with_ss[4]
}

for ticker, weight in simm_weights_with_ss_dict.items():
    print(f"{ticker} -> SIMM Weights with Short Selling in %: ", weight * 100)

VGT -> SIMM Weights with Short Selling in %:  54.34049449837055
XLK -> SIMM Weights with Short Selling in %:  129.77087733342225
SMH -> SIMM Weights with Short Selling in %:  -48.84033375687431
IYT -> SIMM Weights with Short Selling in %:  -5.363098253104647
SOXX -> SIMM Weights with Short Selling in %:  -29.907939821813834


With weights found, we can find the expected return of portfolio and the risk using the formula:

$$
u_p = \alpha_p + \beta_p \cdot u_m

\newline

\sigma_p^2 = \beta_p^2 \cdot \sigma_m^2 + \tau_p^2

\newline

where:
\newline

\alpha_p = \sum_{i=1}^{n} x_i \cdot \alpha_i

\newline

\beta_p = \sum_{i=1}^{n} x_i \cdot \beta_i

\newline

\tau_p^2 = \sum_{i=1}^{n} x_i^2 \cdot \tau_i^2

In [128]:
simm_expected_return_with_ss, simm_portfolio_risk_with_ss = Utils.calculate_factor_model_percentage_portfolio_return_and_risk(simm_weights_with_ss, market_index_data_expected_return, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk with Short Selling in %: ", simm_portfolio_risk_with_ss * 100)
print("Expected Return with Short Selling in %: ", simm_expected_return_with_ss * 100)

Portfolio Risk with Short Selling in %:  1.8471704127716135
Expected Return with Short Selling in %:  66.47697309344825


In [129]:
# find the sharpe ratio
simm_portfolio_mean_return_with_ss = Utils.calculate_portfolio_mean_return(simm_weights_with_ss, mean_daily_return, annualized_factor)
print("Mean Daily Portfolio Return with Short Selling in %: ", simm_portfolio_mean_return_with_ss * 100)

simm_sharpe_ratio_with_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_with_ss, simm_portfolio_risk_with_ss, risk_free_rate)
print("SIMM Sharpe Ratio with Short Selling: ", simm_sharpe_ratio_with_ss)

Mean Daily Portfolio Return with Short Selling in %:  1.1311006076539922
SIMM Sharpe Ratio with Short Selling:  -0.137626330130227


In [130]:
# find optimal weights when short selling is not allowed
simm_weights_no_ss = simm_optimization.find_weights_no_short_selling()

In [131]:
simm_weights_no_ss_dict = {
  "VGT": simm_weights_no_ss[0],
  "XLK": simm_weights_no_ss[1],
  "SMH": simm_weights_no_ss[2],
  "IYT": simm_weights_no_ss[3],
  "SOXX": simm_weights_no_ss[4]
}

for ticker, weight in simm_weights_no_ss_dict.items():
  print(f"{ticker} -> SIMM Weights without Short Selling in %: ", weight * 100)
    

VGT -> SIMM Weights without Short Selling in %:  25.00233001661181
XLK -> SIMM Weights without Short Selling in %:  53.456615797977825
SMH -> SIMM Weights without Short Selling in %:  2.1623705602233008
IYT -> SIMM Weights without Short Selling in %:  17.851539529008946
SOXX -> SIMM Weights without Short Selling in %:  1.5271440961781206


In [132]:
# we will now work to find the expected return and risk of the portfolio
simm_expected_return_without_ss, simm_portfolio_risk_no_ss = Utils.calculate_factor_model_percentage_portfolio_return_and_risk(simm_weights_no_ss, market_index_data_expected_return, market_index_data_variance, alpha_list, beta_list, tau_list)
print("Portfolio Risk without Short Selling in %: ", simm_portfolio_risk_no_ss * 100)
print("Expected Return without Short Selling in %: ", simm_expected_return_without_ss * 100)

Portfolio Risk without Short Selling in %:  0.6865617185253722
Expected Return without Short Selling in %:  95.49844700703719


In [133]:
# find the sharpe ratio
simm_portfolio_mean_return_without_ss = Utils.calculate_portfolio_mean_return(simm_weights_no_ss, mean_daily_return, annualized_factor)
print("Mean Portfolio Return without Short Selling in %: ", simm_portfolio_mean_return_without_ss * 100)

simm_sharpe_ratio_no_ss = Utils.calculate_sharpe_ratio(simm_portfolio_mean_return_without_ss, simm_portfolio_risk_no_ss, risk_free_rate)
print("SIMM Sharpe Ratio without Short Selling: ", simm_sharpe_ratio_no_ss)

Mean Portfolio Return without Short Selling in %:  2.298911578493488
SIMM Sharpe Ratio without Short Selling:  0.3098884948471577
